In [1]:
from neo4j import GraphDatabase
import dotenv
import os

load_status = dotenv.load_dotenv("C:\\Users\\Usuario\\Documents\\Workspace\\Estudio-Psiquiatricos\\code\\neo4j\\Neo4j-921e6a7b-Created-2025-10-13.txt")

if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))


driver = GraphDatabase.driver(URI, auth=AUTH)

In [2]:
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params or {})
        return [record.data() for record in result]

In [7]:
degree_query = """
CALL gds.degree.stream(
  'Diagnostico',
  'DIAGNOSTICO_ASOCIADO'
)
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).terminoIN AS diagnostico, score
ORDER BY score DESC
LIMIT 20
"""

degree_results = run_query(degree_query)
print("Top 20 diagnósticos por Degree Centrality:")
for r in degree_results:
    print(f"{r['diagnostico']}: {r['score']}")

CypherSyntaxError: {neo4j_code: Neo.ClientError.Statement.SyntaxError} {message: Type mismatch: expected Map, Node or Relationship but was String (line 4, column 3 (offset: 44))
"  'DIAGNOSTICO_ASOCIADO'"
   ^} {gql_status: 42001} {gql_status_description: error: syntax error or access rule violation - invalid syntax}

In [8]:
betweenness_query = """
CALL gds.betweenness.stream({
  nodeProjection: 'Diagnostico',
  relationshipProjection: 'TIENE_DIAGNOSTICO'
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).terminoIN AS diagnostico, score
ORDER BY score DESC
LIMIT 20
"""

betweenness_results = run_query(betweenness_query)
print("Top 20 diagnósticos por Betweenness Centrality:")
for r in betweenness_results:
    print(f"{r['diagnostico']}: {r['score']}")

CypherSyntaxError: {neo4j_code: Neo.ClientError.Statement.SyntaxError} {message: Type mismatch: expected String but was Map (line 2, column 29 (offset: 29))
"CALL gds.betweenness.stream({"
                             ^} {gql_status: 42001} {gql_status_description: error: syntax error or access rule violation - invalid syntax}

In [ ]:
eigenvector_query = """
CALL gds.eigenvector.stream({
  nodeProjection: 'Diagnostico',
  relationshipProjection: 'TIENE_DIAGNOSTICO'
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).terminoIN AS diagnostico, score
ORDER BY score DESC
LIMIT 20
"""

eigenvector_results = run_query(eigenvector_query)
print("Top 20 diagnósticos por Eigenvector Centrality:")
for r in eigenvector_results:
    print(f"{r['diagnostico']}: {r['score']}")

In [ ]:
import pandas as pd

df_degree = pd.DataFrame(degree_results)
df_betweenness = pd.DataFrame(betweenness_results)
df_eigenvector = pd.DataFrame(eigenvector_results)

df = df_degree.merge(df_betweenness, on="diagnostico", how="outer", suffixes=("_degree", "_betweenness"))
df = df.merge(df_eigenvector, on="diagnostico", how="outer")
df.rename(columns={"score": "eigenvector"}, inplace=True)

print(df.head())

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(df["score_degree"], df["score_betweenness"])
plt.xlabel("Degree Centrality")
plt.ylabel("Betweenness Centrality")
plt.title("Relación entre Degree y Betweenness Centrality")
plt.show()